In [ ]:
%%time
import asyncio
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()
import json
import pprint
import pandas as pd
import re

In [ ]:
# Fetching data through API request  
r = await asession.get('https://www.zara.com/in/en/categories?ajax=true')
r.status_code

In [ ]:
json_string = r.text
data = json.loads(json_string)
# List to collect all the category codes
my_list = []

In [ ]:
%%time
# creating urls for each page of the site
def invoke(id,name,sec_name,seo_id,extra):
    url = f"https://www.zara.com/in/en/category/{id}/products?regionGroupId=69&ajax=true"
    my_list.append([id,name,sec_name,seo_id, extra,url])


cat = data['categories']    
for i in cat:
    extra = 0
    # 4 main sc_1 (WOMAN, MAN, KIDS, BEAUTY)
    sc_1 = i['subcategories']
    for j in sc_1:
        # BEAUTY has only one sub-section
        id =j['id']
        name = j['name']
        sec_name = j.get("sectionName")
        seo_id = j.get('seo', {}).get('seoCategoryId', None)
        if(i['name'] == 'BEAUTY'):
            invoke(id,name,sec_name,seo_id, extra)
        elif (j['subcategories'] != []):
            sc_2 = j['subcategories']
            for k in sc_2:
                # MAN, WOMAN last subs
                id = k['id']
                m_name = k['name']
                seo_id = k.get('seo', {}).get('seoCategoryId', None)
                invoke(id,m_name,sec_name,seo_id,extra)
                if (k['subcategories'] != []):
                    # KIDS collection subcategories
                    sc_3 = k['subcategories']
                    for k_sc in sc_3:                                        
                        id = k_sc['id']
                        name = k_sc['name']
                        seo_id = k_sc.get('seo', {}).get('seoCategoryId', None)
                        invoke(id,name,sec_name,seo_id,extra=m_name)        

In [ ]:
%%time
len(my_list)
df = pd.DataFrame(my_list)
df.to_csv('zara_code_list')

In [ ]:
%%time
#improve
regex = r'\+ Info|JOIN LIFE|GIFT CARD|info'
for i in my_list:
    for j in i:
        if isinstance(j, str) and bool(re.search(regex, str(j), re.IGNORECASE)):
            my_list.remove(i)

print(len(my_list))

In [ ]:
sections = ['WOMAN', 'MAN', 'KID', 'BEAUTY']
product_list = []
links = []

for i in sections:
    print(i)
    for j in my_list:        
        if i == j[2]:
            val = j[5]
            c = j[0]
            # Collecting all responses of the urls
            r = await asession.get(val)
            print(r.status_code)
            try:
                r.raise_for_status()  # Raise an error for bad HTTP responses
                json_string = r.text
                data = json.loads(json_string)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
            except Exception as e:
                print(f"HTTP request error: {e}")
            product_list.append(data)
            links.append(c)     

In [ ]:
import os


output_directory = "C:\Json files of zara 1200+responses"

os.makedirs(output_directory, exist_ok=True)

# Save each JSON object in a separate file
for i, json_data in enumerate(product_list):
    file_path = os.path.join(output_directory, f'zarajson_{i}_{links[i]}.json')
    with open(file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)

print(f"All {len(product_list)} JSON files have been saved to {output_directory}")

In [ ]:
%%time

def call(i):
    file_path = f"C:\Json files of zara 1200+responses\zarajson_{i}_{links[i]}.json"
    
    # Read the JSON file
    with open(file_path, 'r') as json_file:
        response = json.load(json_file)    
    # Process the JSON data as needed
    print(f"Processing {file_path}:")
    return response

# parsing every response and then storing them into individual files
def extract_info_recursive_generator(data):
    if isinstance(data, list):
        for item in data:
            yield from extract_info_recursive_generator(item)
    elif isinstance(data, dict):
        xmedia = data.get("commercialComponents", {})
        for entry in xmedia:
            yield {
                "name": entry.get("name"),
                "description": entry.get("description"),
                "price": entry.get("price"),
                "sectionName": entry.get("sectionName"),
                "familyName": entry.get("familyName"),
                "subfamilyName": entry.get("subfamilyName"),
                "id": entry.get("detail", {}).get("colors", [])[0].get("id"),
                "productId": entry.get("detail", {}).get("colors", [])[0].get("productId"),
                "color": entry.get("detail", {}).get("colors", [])[0].get("name")
            }

        for value in data.values():
            yield from extract_info_recursive_generator(value)

for i in range(1194):
    data = call(i)
    extracted_info_generator = extract_info_recursive_generator(data)
    extracted_info_list = list(extracted_info_generator)

    output_directory = "C:\Zara product details"
    os.makedirs(output_directory, exist_ok=True)

    # Save the extracted information to a separate JSON file for each input file
    output_file_path = os.path.join(output_directory, f"parsed_output_{i}.json")
    with open(output_file_path, 'w') as output_file:
        json.dump(extracted_info_list, output_file)


In [ ]:
# Processing every output file and combining all of it into one single large dataframe
result_df = pd.DataFrame()
for i in range(1194):        
    # Specify the directory where the JSON files are saved
    file_path = f"C:\Zara product details\parsed_output_{i}.json"
    # Read the JSON file
    with open(file_path, 'r') as json_file:
        response = json.load(json_file)    
    # Process the JSON data as needed
    print(f"Processing {file_path}:")
    df = pd.DataFrame(response)
    df['category_id'] = links[i]
    result_df = pd.concat([result_df, df], ignore_index=True, sort=False)

In [ ]:
# Total rows = 102147
result_df.tail(30)